In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('./data/final_stats.pickle', 'rb') as final_stats:
    stats = pickle.load(final_stats)

In [ ]:
stats[2001]

In [ ]:
with open('./data/cleaned_data/players_stats.pickle', 'rb') as play_stats:
    player_stats = pickle.load(play_stats)

In [ ]:
player_stats[2001]

In [1]:
import pandas as pd
import pickle
from unidecode import unidecode

In [2]:
years = list(range(2001, 2024))

In [3]:
distinct_players = pd.read_csv("./data/cleaned_data/distinct_players.csv", encoding='utf-8')

In [4]:
salary_cap = pd.read_csv('./data/cleaned_data/salary_cap.csv', encoding='utf-8')

In [ ]:
distinct_players_not_accented = distinct_players.copy()
distinct_players_not_accented['player'] = distinct_players['player'].apply(lambda x: unidecode(x))

In [ ]:
with open('./data/cleaned_data/players_stats.pickle', 'rb') as play_stats:
    players_stats = pickle.load(play_stats)

In [ ]:
players_not_found = []
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())

    with open(f'./data/raw_data/players_salaries/players_salaries_{year}.csv', 'r', encoding="UTF-8") as file:
        players_salaries = pd.read_csv(file, delimiter=';')

    players_salaries.rename(columns = {'PLAYER': 'ID', f'{year-1}/{str(year)[2:]}': 'salary'}, inplace = True)
    players_salaries['salary'] = players_salaries['salary'].apply(lambda x: int(str(x)[1:].replace(",", "")))
    players_salaries

    for index, row in players_salaries.iterrows():
        row_id = row['ID']
        try:
            players_salaries.loc[index, 'ID'] = distinct_players_not_accented.query(f'player == "{row_id}"')["ID"].iloc[0]
        except:
            players_not_found.append(str(players_salaries.loc[index, 'ID']))

players_not_found

In [ ]:
with open(f'./data/raw_data/players_salaries/players_salaries_{year}.csv', 'r', encoding="UTF-8") as file:
        players_salaries = pd.read_csv(file, delimiter=';')

In [ ]:
players_salaries.rename(columns = {'PLAYER': 'ID', f'{year-1}/{str(year)[2:]}': 'salary'}, inplace = True)
players_salaries['salary'] = players_salaries['salary'].apply(lambda x: int(str(x)[1:].replace(",", "")))
players_salaries

In [ ]:
for index, row in players_salaries.iterrows():
    row_id = row['ID']
    try:
        players_salaries.loc[index, 'ID'] = distinct_players_not_accented.query(f'player == "{row_id}"')["ID"].iloc[0]
    except:
        print(players_salaries.loc[index, 'ID'])

In [ ]:
for player_id in player_list:    
    try:
        salary = players_salaries.query(f"ID == '{player_id}'")["salary"].iloc[0]
        cap = salary_cap.query(f"year == {year}")["salary_cap"].iloc[0]
        salary_perc = round(salary / cap, 2)
    except:
        print(f"##### {player_id} | {salary_perc} #####")